In [353]:
import pandas as pd
import re

### Hypertension 관련 ICD-9 CODE 가져오기
##### -> "hypertension_list"에 총 56개의 ICD-9 CODE 저장

In [355]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/D_ICD_DIAGNOSES.csv")
# df = pd.read_csv("~/project/MIMIC-III/D_ICD_DIAGNOSES.csv")

In [356]:
# DIAGNOSIS 데이터의 LONG_TITLE 칼럼에서 "hypertension"이라는 단어가 등장하는 경우
df1 = df[df['LONG_TITLE'].str.contains('hypertension')]
print(len(df1))

56


In [357]:
hypertension_list = df1["ICD9_CODE"].to_list()
print(len(hypertension_list))

56


### Hypertension 진단 받은 SUBJECT ID 선별
##### -> "subject_list"에 18,871개 SUBJECT ID 저장

In [358]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/DIAGNOSES_ICD.csv")

In [359]:
def add(group):
    return ' '.join(group['ICD9_CODE'].fillna(""))

df1 = df.groupby(['SUBJECT_ID', 'HADM_ID']).apply(add).reset_index(name="ICD9_CODE")

In [360]:
mask = df1['ICD9_CODE'].str.contains('|'.join(hypertension_list), na=False)
df2 = df1[mask]

In [363]:
# 동일한 환자의 경우 최초 진단 기록만 사용하기 위해 admission table 입원 시간 기록 데이터 가져와서 합치기
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/ADMISSIONS.csv")
df3 = df[["HADM_ID", "ADMITTIME"]]
df4 = pd.merge(df2, df3, on="HADM_ID")
df4['ADMITTIME'] = pd.to_datetime(df4['ADMITTIME'])
# df4.head(10)

In [364]:
df5 = df4.sort_values(by=['SUBJECT_ID', 'ADMITTIME'], ascending=[True, True])
df5 = df5.drop_duplicates(subset=['SUBJECT_ID'], keep='first')
df5.sort_values(by=["SUBJECT_ID"], ascending=True, inplace=True)
# df5.head(10) # 7,8번 index의 값 중 ADMITTIME이 빠른 index만 남은 것을 확인할 수 있다.

In [365]:
subject_list = df5["SUBJECT_ID"].to_list()
print(len(subject_list))

18871


### Hypertension 진단 받은 환자 사망률 확인
-> 전체 18,871명 중 생존(0)은 11,412명, 사망(1)은 7459명으로 6:4 정도의 비율 <p>
-> 성별에 따른 비는 남자 10,525명, 여자 7,459명으로 5.5:4.5 정도의 비율

In [366]:
df = pd.read_csv("/data/PUBLIC_DATA/MIMIC-III/PATIENTS.csv")

In [367]:
df1 = df[df["SUBJECT_ID"].isin(subject_list)]

In [368]:
# Hypertension 진단 받은 사람의 사망 비율
df2 = df1["EXPIRE_FLAG"].value_counts().to_frame(name="Count")
df2["ratio"] = df1["EXPIRE_FLAG"].value_counts(normalize=True)
df2

,Count,ratio
0,11412,0.604737
1,7459,0.395263


In [369]:
# Hypertension 진단 받은 사람의 성별 비율
df3 = df1["GENDER"].value_counts().to_frame(name="Gender")
df3["ratio"] = df1["GENDER"].value_counts(normalize=True)
df3

,Gender,ratio
M,10525,0.557734
F,8346,0.442266


In [370]:
# Hypertension 진단 받은 사람의 성별에 따른 사망 비율
df4 = df1.groupby(["GENDER", "EXPIRE_FLAG"]).size().unstack(fill_value=0)
df4["SUM"] = df1["GENDER"].value_counts()
df4

EXPIRE_FLAG,0,1,SUM
GENDER,,,
F,4693,3653,8346
M,6719,3806,10525
